In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

In [2]:
print(torch.__version__)

1.3.0


In [3]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [4]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=60)
        self.out = nn.Linear(in_features=60,out_features=10)
        
    def forward(self,t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
        t = t.reshape(-1,12*4*4)
        t = F.relu(self.fc1(t))
        
        t = F.relu(self.fc2(t))
        
        t = self.out(t)
        
        return t
    

In [6]:
tran_set = torchvision.datasets.FashionMNIST(
        root='./data',
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor()
        ])
)

0it [00:00, ?it/s]

100%|█████████▉| 26419200/26421880 [01:22<00:00, 380768.04it/s] 

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



0it [00:00, ?it/s]


 83%|████████▎ | 24576/29515 [00:00<00:00, 65391.06it/s]
32768it [00:01, 29153.15it/s]                           

0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



 95%|█████████▌| 4210688/4422102 [00:04<00:00, 1453410.98it/s]
4423680it [00:04, 970527.51it/s]                              

0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



8192it [00:00, 22092.23it/s]            

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw
Processing...
Done!


In [10]:
network = Network()

train_loader = torch.utils.data.DataLoader(tran_set,batch_size=100)
optimizer = optim.Adam(network.parameters(),lr=0.01)

for epoch in range(5):
    total_loss =0
    total_correct = 0
    for batch in train_loader:
        images,labels = batch

        preds = network(images)#pass Batch
        loss = F.cross_entropy(preds,labels) #calculte loss

        optimizer.zero_grad() 
        loss.backward() #calculate Grads
        optimizer.step()#updata weights

        total_loss += loss.item()
        total_correct+=get_num_correct(preds,labels)
    print("epoch:",epoch,"total correct:",total_correct,"loss:",total_loss)

epoch: 0 total correct: 46502 loss: 358.44425243139267
epoch: 1 total correct: 51063 loss: 241.92368979752064
epoch: 2 total correct: 51704 loss: 221.7484328597784
epoch: 3 total correct: 52103 loss: 214.53608672320843
epoch: 4 total correct: 52046 loss: 210.90614467859268


In [11]:
total_correct/len(tran_set)

0.8674333333333333